In [ ]:
# notebook

# print full np arrays
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

# import libraries

import rasterio
import rasterio.plot
from rasterio.crs import CRS
from matplotlib import pyplot as plt
import geopandas as gpd
import pandas as pd
import numpy as np
from tqdm import tqdm
import rasterio.mask
from shapely.geometry import Point
from geopandas import GeoDataFrame
import matplotlib.colors as colors
from matplotlib import cm
from shapely.geometry import Polygon
from rasterio.transform import from_gcps
from rasterio.control import GroundControlPoint as GCP
import osmnx
from pathlib import Path

from sklearn.metrics import (
    root_mean_squared_error,
    mean_squared_error,
    mean_absolute_error,
)

plt.rcParams.update({'font.size': 20})
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

from tqdm.auto import tqdm
tqdm.pandas()

from utils import *
from map_utils import *

In [ ]:
# set vars here! to define which map is created in which way

region = "germany"
map_path = f"intermediate/map_{region}.tif"

ROADS = False
CITIES = False
SPOTS = False
CIRCLE = 50000 # circle around spot in m for which we assume the waiting time at the spot is definitely relevant
DAY = 24 * 60
WAIT_MAX = DAY
# resolution 40 for world -> 6800x14400 image -> lets calculating distribution crash -> too many values in np array to calculate?

THRESHOLD = 0.00000001 # sum of distributions for at pixel at which we assume we have no data there; quite arbitrary
RECOMPUTE = True # otherwise load from cache if possible

# color the whole map smoothly & fine grained where possible

# approach 1
# adjust stdv of single points
DYNAMIC = False
K = 1
FACTOR_STDV = 1 # the xth stdv

# approach 2
# color not defined areas iteratively by using larger stdvs
ITERATIVE = False
ITERATIONS = 10



In [ ]:

def get_points(path):
    points = gpd.read_file(path)
    points.wait = points.wait.astype(float)
    points.lat = points.lat.astype(float)
    points.lon = points.lon.astype(float)
    # threshold - assuming that values above that are skewed due to angriness/ subjectivity of the hiker
    points = points[points['wait'] <= WAIT_MAX]

    # use epsg 3857 as default as it gives coordinates in meters
    points.geometry = gpd.points_from_xy(points.lon, points.lat)
    points.crs = CRS.from_epsg(4326)
    points = points.to_crs(epsg=3857)

    return points
    
# read from the hitchmap dump
points = get_points('../data/points_train.csv')
all_points = points

# to get the same example as here https://abelblogja.wordpress.com/average-waiting-times-in-europe/
artificial_points = get_points("../data/artificial_points.csv")

In [ ]:
points, polygon, map = get_points_in_region(points, region)
if region == "artificial": 
        points = artificial_points

In [ ]:
# set radius of all points +x km and recompute to only color remaining "not-enough-data" pixels
# -> repeat untill full map is colored
if ITERATIVE:
    for i in range(0, ITERATIONS):
        X, Y, Z_raw, Zn, Zn_w = make_raster_map(points, polygon, map, i)

        if i == 0:
            Z = Z_raw
        else:
            Z = np.where(np.logical_and(Z_raw != -1.0, Z == -1.0), Z_raw, Z)

    save_raster(Z, polygon, map)
else:
    X, Y, Z_raw, Zn, Zn_w = make_raster_map(points, polygon, map)
    Z = Z_raw
    save_as_raster(Z, polygon, map, map_path)

In [ ]:
# preview - where the single distributions are placed
plt.contourf(X, Y, np.clip(Z,0,200))
plt.colorbar()
plt.show()

In [ ]:
states = gpd.read_file("map_features/states/ne_10m_admin_1_states_provinces.shp")
states = states.to_crs(epsg=3857)

# use smaller units for Russia
# country level except for Canada, Russia, USA, Australia, China, Brazil, India, Indonesia
states = states[states.admin != "Antarctica"]

# a state is hitchhikable if there are hitchhiking spots in it
def check_hitchhikability(state):
    points_in_state = points[points.geometry.within(state.geometry)]
    return len(points_in_state) > 0

states["hh"] = states.progress_apply(check_hitchhikability, axis=1)

In [ ]:
# define the heatmap color scale

# TODO smoother spectrum instead of buckets
buckets = [
    'grey', # not enough data
    '#008200', # dark green
    '#00c800', # light green
    'green', # green
    '#c8ff00', # light yellow
    '#ffff00', # yellow
    '#ffc800', # light orange
    '#ff8200', # dark orange
    'red', # red
    '#c80000', # dark red
    '#820000', # wine red
    'blue', # not necessary to color
    ]

cmap = colors.ListedColormap(buckets)

max_wait = all_points.wait.max() + 0.1 # to get at least this value as maximum for the colored buckets
num_scale_colors = len(buckets) - 2 # because of upper and lower bucket
# build log scale starting at 0 and ending at max wait
base = (max_wait + 1) ** (1/num_scale_colors)

def log_scale(x):
    return base ** x - 1

# how to prevent numerical instabilities resulting in some areas having a checkerboard pattern
# round pixel values to ints
# set the boundaries of the buckets to not be ints
    # should happen automatically through the log scale
    # -0.1 should be 0.0 actually
# boundary of last bucket does not matter - values outside of the range are colored in the last bucket
boundaries = [-1, -0.1] + [log_scale(i) for i in range(1, num_scale_colors+1)] + [max_wait + 1]

# prepare the plot
norm = colors.BoundaryNorm(boundaries, cmap.N, clip=True)
fig, ax = plt.subplots(figsize=(100, 100))


# get borders of all countries
# download https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/110m/cultural/ne_110m_admin_0_countries.zip
# from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/
countries = gpd.datasets.get_path("naturalearth_lowres")
countries = gpd.read_file(countries)
countries = countries.to_crs(epsg=3857)
countries = countries[countries.name != "Antarctica"]
# TODO so far does not work as in final map the raster is not applied to the whole region
# countries = countries[countries.geometry.within(polygon.geometry[0])]
country_shapes = countries.geometry
countries.plot(ax=ax, facecolor='none', edgecolor='black')

# TODO takes more time than expected
# use a pre-compiled list of important cities
# download https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/ne_10m_populated_places.zip
# from https://www.naturalearthdata.com/downloads/10m-cultural-vectors/
# cities = gpd.read_file("cities/ne_10m_populated_places.shp", bbox=polygon.geometry[0]) should work but does not
if CITIES:
    cities = gpd.read_file("map_features/cities/ne_10m_populated_places.shp") # takes most time
    cities = cities.to_crs(epsg=3857)
    cities = cities[cities.geometry.within(polygon.geometry[0])]
    cities.plot(ax=ax, markersize=1, color='black')

# use a pre-compiles list of important roads
# download https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/ne_10m_roads.zip
# from https://www.naturalearthdata.com/downloads/10m-cultural-vectors/
if ROADS:
    roads = gpd.read_file("roads/ne_10m_roads.shp")
    roads = roads.to_crs(epsg=3857)
    roads = roads[roads.geometry.within(polygon.geometry[0])]
    roads.plot(ax=ax, markersize=1, color='black')

if SPOTS: all_points.plot(ax=ax, markersize=10, color='red')

# limit heatmap to landmass by asigning inf/ high value to sea
with rasterio.open(map_path) as heatmap:
    out_image, out_transform = rasterio.mask.mask(heatmap, country_shapes, crop=True, filled=False)
    out_meta = heatmap.meta
    
out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

with rasterio.open(map_path, "w", **out_meta) as destination:
    destination.write(out_image)


# plot the heatmap
raster = rasterio.open(map_path)
rasterio.plot.show(raster, ax=ax, cmap=cmap, norm=norm)

fig.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap), ax=ax)
if ITERATIVE: file_name = f"maps/map_{region}_iter_{ITERATIONS}.png"
elif DYNAMIC: file_name = f"maps/map_{region}_{K}.png"
else: file_name = f"maps/map_{region}.png"
plt.savefig(file_name, bbox_inches='tight')

# eval

In [ ]:
points['lat'] = points.geometry.y
points['lon'] = points.geometry.x

points["pred"] = points.progress_apply(lambda x: map_predict(x.lon, x.lat, raster), axis=1)

no_data = 1e20 # ocean area
points = points[points['pred'] != no_data]

print((mean_squared_error(points["wait"], points["pred"]), 
root_mean_squared_error(points["wait"], points["pred"]),
mean_absolute_error(points["wait"], points["pred"])))

In [ ]:
val = get_points('../data/points_val.csv')
val, polygon, map_boundary = get_points_in_region(val, region)

val['lat'] = val.geometry.y
val['lon'] = val.geometry.x

# X = val[['lat', 'lon']].values
val['pred'] = val.progress_apply(lambda x: map_predict(x.lon, x.lat, raster), axis=1)

# for val points in the sea or on islands the model does not provide valid predictions
no_data = 1e20 # ocean area
val = val[val['pred'] != no_data]

(mean_squared_error(val['wait'], val['pred']),
root_mean_squared_error(val['wait'], val['pred']),
mean_absolute_error(val['wait'], val['pred']))

In [ ]:
# germany (2017.0828210097397, 44.91194519289651, 26.35369707811568)
# europe 